In [1]:
import os, sys
import numpy as np
from sklearn.model_selection import KFold
from matplotlib import pyplot as plt
import spectral

In [2]:
from spectral.algorithms import spectral_angles

In [3]:
x = np.random.rand(1, 10, 25)
endmember = np.random.rand(5, 25)
spectral_angles(x, endmember).shape

(1, 10, 5)

# Data Preprocessing

## Calibration 

$\begin{align}
    C_i = 100 * \frac{R_i - D_r}{W_r - D_r}
\end{align}$

where $C_i$ is the calibrated image, $R_i$ note raw image and the $W_r$ and $D_r$ represents the white and dark reference image, respectively.


In [4]:
# W_r y D_r es una imagen
def calibrate(img, w_r, d_r):
    if not(w_r.shape == d_r.shape == img.shape):
        assert('Dimensionality error')
    
    return 100  * (img - d_r) / (w_r - d_r)

# Normalize data
$\begin{align}
    P'_i = \frac{P_i - P_{min}}{P_{max} - P_{min}}
\end{align}$

where $P'_i$ is the normalized pixel value, $P_i$ the reflectance of the pixel, $P_{min}$ and $P_{max}$ is the minimum and maximum reflectance value, respectively.

In [5]:
def normalize(img):
    return (img - img.min()) / (img.max() - img.min())

# Sampling Interval Analysis

Selected band equidistant...

$\begin{align}
    Sampling Interval (nm) = \frac{\lambda_{max} - \lambda_{min}}{N_{\lambda}}
\end{align}$

where $\lambda_{max} - \lambda_{min}$ is the difference between the mamum and minimum wavelength and $N_{\lambda}$ is the number of band captured by the sensor.

In [6]:
def sampling_interval(lambda_min, lambda_max, n_spectral_bands):
    '''
        Param:
        -----
            lambda_min (int): minimum wavelenght
            lambda_max (int): maximum wavelenght
            n_spectral_bands (int): number of spectral bands captured by the sensor
    '''
    return (lambda_max - lambda_min) / n_spectral_bands

# Dataset reduction

**Spectral Angle Mapper**

$[\alpha = cos^{-1}\left ( \frac{\sum_{i = 1}^{nb} t_{i} r_{i}}{(\sum_{i = 1}^{nb} t_{i}^2)^{\frac{1}{2}} (\sum_{i = 1}^{nb} r_{i}^2)^{\frac{1}{2}}} \right )]$

where

* $\alpha$ = spectral angle between the standard and the spectral curve of the pixel
* $nb$ = number of spectral channels
* $t$ = vector of spectral response of the standard
* $r$ = the spectral response vector of the analyzed pixel

In [8]:
from sklearn.cluster import KMeans
from spectral.algorithms import spectral_angles

# example, 12 pixels with 125 bands
X = np.random.rand(12, 125)

kmeans = KMeans(n_clusters=4, random_state=0).fit(X)
centroid = kmeans.cluster_centers_

spectral_angles(X[0:1].T, centroid[0:1])

IndexError: tuple index out of range

In [22]:
X = np.random.rand(3, 125)
result = spectral_angles(X[np.newaxis,:], X)

print(result.shape)

print(result)

(1, 3, 3)
[[[1.49011612e-08 7.44227300e-01 6.83935871e-01]
  [7.44227300e-01 0.00000000e+00 6.96270566e-01]
  [6.83935871e-01 6.96270566e-01 0.00000000e+00]]]


# Ant Colony Optimization

probar **scikit-opt**: https://github.com/guofei9987/scikit-opt

https://www.youtube.com/watch?v=YFN_fJEu63w

In [25]:
nodes = []
for _ in range(20):
  x = np.random.uniform(-10, 10)
  y = np.random.uniform(-10, 10)
  nodes.append((x, y))

nodes

[(6.4200452281555656, 8.401774049084544),
 (-0.46999706371519956, 7.88905580497774),
 (-6.265259712404678, 1.6056087735621674),
 (6.50101159805244, -6.824581606638129),
 (0.5955927677020671, -7.087736474899655),
 (-1.6332397654825996, -1.2495346791678443),
 (-7.9160886235093635, -7.508991081808867),
 (-5.811732899313027, 1.5561341777805247),
 (3.6864347218229874, 7.162074747305994),
 (5.743206401666143, -6.609895212112398),
 (-9.04064865258552, 5.8123433793520185),
 (-9.178067762674294, 8.621554260206608),
 (8.098376012945934, 5.0296021591423195),
 (7.151823267243945, 4.1413139383830995),
 (6.229061732948427, -4.038411445300469),
 (5.773269734278868, 7.013628100370163),
 (-2.620418184558404, 1.3326897732255247),
 (-6.843800602316657, -6.100545852700165),
 (-7.84394659039169, 9.770872571122705),
 (3.892656287809624, 2.2968522483029847)]

In [59]:
import pants
import math

def euclidean(a, b):
    return math.sqrt(pow(a[1] - b[1], 2) + pow(a[0] - b[0], 2))

In [72]:
world = pants.World(nodes, euclidean)
solver = pants.Solver()
# solution = solver.solve(world)

solutions = solver.solutions(world)


In [73]:
# print(solution.distance)
# print(solution.tour)    # Nodes visited in order
# print(len(solution.tour))
# print(solution.path)    # Edges taken in order
# print(len(solution.path))

best = float("inf")
for solution in solutions:
    assert solution.distance < best
    best = solution.distance
    print(best)
    print(len(solution.path))

print(best)

101.56545346105301
20
91.66989068189157
20
89.73962085507011
20
85.688038102191
20
83.7715069452407
20
82.72891918212392
20
74.90665778119947
20
73.41599938461654
20
73.41599938461654


For each centroid, only the 10 most similar pixels are selected, having a total of 1000 pixels per class (100 centroids ×10 pixels). Thus, the reduced dataset is
intended to avoid the inclusion of redundant information in the training of the supervised classifier...

In [77]:
solver.trace_elite[0]

TypeError: 'method' object is not subscriptable

In [ ]:
n_cluster = 2

X = np.array([[1, 2.5], [1, 4.1], [1, 0.1],
        [10, 2.1], [10, 4.9], [10, 0]])

kmeans = KMeans(n_clusters=n_cluster, random_state=0).fit(X)
centroid = kmeans.cluster_centers_
for x in X:
    plt.scatter(x=x[0], y=x[1], alpha=.2)

for x in centroid:
    plt.scatter(x=x[0], y=x[1], marker='*')

plt.show()

# Spectral Angle Mapper

$[\alpha = cos^{-1}\left ( \frac{\sum_{i = 1}^{nb} t_{i} r_{i}}{(\sum_{i = 1}^{nb} t_{i}^2)^{\frac{1}{2}} (\sum_{i = 1}^{nb} r_{i}^2)^{\frac{1}{2}}} \right )]$

where

* $\alpha$ = spectral angle between the standard and the spectral curve of the pixel
* $nb$ = number of spectral channels
* $t$ = vector of spectral response of the standard
* $r$ = the spectral response vector of the analyzed pixel

In [ ]:
X = ["a", "b", "c", "d"]
kf = KFold(n_splits=4)
for train, test in kf.split(X):
    print("%s %s" % (train, test))